In [2]:
from __future__ import annotations
# Assistive imports
from collections import namedtuple
from typing import Tuple, List, Optional, Set, Union, Generator
from heapq import heappush, heappop
from collections import deque
from functools import partial


import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.animation as animation

In [4]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds)
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [5]:
NewState = namedtuple('NewState', ['cost', 'coordinates', 'action'])

In [6]:
# Use the following Node class to generate search tree
import math

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(
            state=state,
            parent=parent,
            action=action,
            path_cost=path_cost
        )
    
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 
    
    # newly defined to assist with trace, and repr redefined
    # for readability
    def __repr__(self):
        """
        Newly defined to assist with trace, and track
        available states throughout the document
        """
        return '<state:(x:{},y:{}) path_cost:{} action:{}>'\
                .format(self.state[0], self.state[1], self.path_cost, self.action)
    
    def __eq__(self, other: Node) -> bool:
        """
        Tests for equality between two instances of a Node.
        Nodes with different states or different path costs 
        are now defined to be treated seperately.
        Args:
            other (Node): 
                Node to compare with (implicitly executed)
                by python when doing comparisons such as node1 == node2
        """
        return (self.state == other.state 
                and self.path_cost == other.path_cost)
    
    def expand_node(self, state: NewState) -> Node:
        """
        Expands on a single state. Method takes a new state and expands 
        by instantiating a new node instance and assigning the 
        costs as the costs so far, and saving the action taken from
        parent_node -> child_node as child.action.
        
        Args:
            state (NewState): 
                Expects a namedtuple as defined in the cell above.
            
        Returns:
            Node: Expanded child node
        """
        expanded_node = Node(state.coordinates)
        expanded_node.parent = self
        expanded_node.path_cost = self.path_cost + state.cost
        expanded_node.action = state.action
        return expanded_node
    
    def expand(
            self,     
            permissable_actions: List[NewState]
        ) -> List[Node]:
        """
        Args:
            permissable_actions (List[NewState]):
                List of possible actions to take
        
        Returns:
            List[Node]:
                List of child nodes for this node.
        """
        return list(map(self.expand_node, permissable_actions))
    
    def __bool__(self):
        """
        Assist with assigning truthyness to the Node class
        e.g. is node == True if state is truthy
        """
        return True if self.state else False
    
    def __hash__(self):
        """
        Allows this class to be used in objects that require
        hashes such as sets, keys in dicts.
        """
        return hash(self.state)


In [7]:
class Maze(Problem):
    def __init__(self,
                 initial: Node,
                 goal: Node,
                 boundaries: Tuple[int, int],
                 action_cost_map: Optional[dict],
                 **kwds):
        """
        Add type hints and parameter to know boundaries
        given the assumption "Assume that the agent knows
        the boundaries of the maze and has full observability"
        
        Args:
            initial (Node): Node for the initial state
            goal (Node): Node for the goal state
            boundaries (Tuple[int,int]):
                (rows, cols) of boundaries, assuming (0,0) to (rows,cols) 
                as problem space
        """
        super().__init__(initial=initial,
                         goal=goal,
                         boundaries=boundaries,
                         action_cost_map=action_cost_map,
                         **kwds)
    
    def action_cost(self, node: Node, action: str) -> int:
        """
        Args:
            node (Node): Current node state
            action (str): Action to take
        
        Returns:
            int: Cost (s, a, s')
        """
        return self.action_cost_map[action]
    
    def _transform_permissable_action(self,
                                      actions: Tuple[Tuple[int, int], str],
                                      node: Node) -> Node:
        state, action = actions
        action_cost = self.action_cost(node, action)
        return NewState(action_cost, state, action)
        
    def actions(self, node: Node) -> List[NewState]:
        """
        Return permissable actions as list of actions
        as (COST, s', ACTION)
        Args:
            node (Node): Agents current state node
        
        Returns:
            List[NewState]:
                List of permissable states and actions to expand to
                for the given node.
        """
        x = node.state[0]
        y = node.state[1]
        parent = node.parent.state if node.parent else None
        
        permissable_x_y = {((x+1, y), 'UP'),
                           ((x-1, y), 'DOWN'),
                           ((x, y+1), 'RIGHT'),
                           ((x, y-1), 'LEFT')}
        
        return list(
                    map(
                        lambda actions: self._transform_permissable_action(actions, node),
                        filter(
                                lambda actions:
                                   # Ensuring they permissable actions are:
                                   # * within boundaries
                                   # * not a shaded region
                                   0<=actions[0][0]<self.boundaries[0] and
                                   0<=actions[0][1]<self.boundaries[1] and
                                   (actions[0][0], actions[0][1]) not in self.shaded_regions,
                               permissable_x_y
                        )
                )
           )
    
    def h(self, node: Node) -> Union[float, int]:
        """
        Implementing Manhatten distance.
        One of two of the heuristics for Q1.a
        
        Args:
            node (Node): Current agent node
        
        Returns:
            Union[float, int]: heuristic value
        """
        return abs(self.goal.state[0]-node.state[0]) + \
                abs(self.goal.state[1]-node.state[1])
    
    def h2(self, node: Node) -> Union[float, int]:
        """
        Implementing Euclidean distance
        One of two of the heuristics for Q1.a
        Args:
            node (Node): Current agent node
        
        Returns:
            Union[float, int]: heuristic value
        """
        return ((self.goal.state[0]-node.state[0])**2 + 
                (self.goal.state[1]-node.state[1])**2)**(1/2)
        
        
    def __repr__(self):
        """
        Assigning a readable representation of the object.
        """
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    
    def is_cycle(self, node: Node) -> bool:
        """
        Args:
            node (Node): Checks if a node is cyclic along its parent nodes
        
        Returns:
            bool: True if it is cyclic, else False.
        """
        visited = set()
        state = node.state
        while node.parent:
            if node.parent.state == state:
                return True
            node = node.parent
        return False
    
    def is_goal(self, node) -> bool:
        """
        Args:
            node (Node):
                Helper function to check if a certain node is the goal
                node based on the parameters of this maze.
        
        Returns: 
            bool:
                True if it is the goal node, false if not
        """
        return node.state == self.goal.state


In [9]:
start_node = Node((8,10))
goal_node = Node((11,9))

In [10]:
maze = Maze(start_node,
            goal_node,
            boundaries=(16, 24),
            action_cost_map={'LEFT': 10, 'RIGHT': 10, 'UP': 1, 'DOWN': 1},
            shaded_regions = {(7, 9),
                              (6, 9),
                              (10, 12),
                              (10, 13),
                              (11, 12),
                              (10, 9),
                              (12, 10),
                              (9, 9),
                              (13, 10),
                              (8, 9),
                              (11, 13),
                              (10, 10),
                              (14, 9),
                              (11, 10)})

In [11]:
maze.initial

<state:(x:8,y:10) path_cost:0 action:None>

In [12]:
def print_stats(node: Node,
                expanded: List[Tuple[int, int]],
                max_frontier_size: int,
                explored_count: int) -> None:
    print((f'Max frontier: {max_frontier_size}'
           f'\nExpanded: {len(expanded)}'
           f'\nPath Cost: {node.path_cost}'
           f'\nExplored nodes: {explored_count}'))

algorithm_results = {}
def save_results(algorithm:str,
                node: Node,
                expanded: List[Tuple[int, int]],
                max_frontier_size: int,
                explored_count: int) -> None:
    """
    Args:
        algorithm (str): Algorithm name
        node (Node): solution node
        expanded (List[Tuple[int, int]]): Expanded nodes
        max_frontier_size (int): Maximum frontier size for the algo
        explored_count (int): Nodes explored for the algo
    
    Updates:
        algorithm_results
    """
    algorithm_results[algorithm] = {
        'solution_node': node,
        'expanded': expanded,
        'max_frontier_size': max_frontier_size,
        'explored_count': explored_count
    }

In [15]:
import sys
visited = set() #size of it is generated nodes#
expanded = []
maximum_frontier_size = 0
nodes_generated = 0
path_cost = sys.maxsize
def depth_first_search(
        curnode: Node,
        maze: Maze ,
        current_frontier_size : int
    ) -> bool:
    """
    Args:
        maze (Maze): Maze class to run the search on
    
    Returns:
        solution (Node):
            Node for which the solution was found
        expanded_nodes (List[Tuple[int, int]]):
            Nodes for which was visited
        maximum_frontier_size (int):
            Maximum frontier size of this algo
    """
    global maximum_frontier_size
    global path_cost
    global nodes_generated
    visited.add(curnode.state)
    if maze.is_goal(curnode):
        if path_cost > curnode.path_cost:
            path_cost = curnode.path_cost
        return
    actions = maze.actions(curnode)
    if current_frontier_size > maximum_frontier_size:
        maximum_frontier_size = current_frontier_size
    for child in curnode.expand(actions):
        if child.state not in visited:
            #print (child , current_frontier_size , path_cost)
            nodes_generated += 1
            depth_first_search(child, maze , 1 + current_frontier_size)
    expanded.append(curnode)
nodes_generated = 1
depth_first_search(maze.initial, maze, 0)
print(len(expanded))
print(maximum_frontier_size)
print(path_cost)
print(nodes_generated)

369
189
167
370


In [19]:
def breadth_first_search(
        maze: Maze
    ) -> (Node, List[Tuple[int, int]], int):
    """
    Args:
        maze (Maze): Maze class to run the search on
    
    Returns:
        solution (Node):
            Node for which the solution was found
        expanded_nodes (List[Tuple[int, int]]):
            Nodes for which was visited
        maximum_frontier_size (int):
            Maximum frontier size of this algo
    """
    initial_node = maze.initial
    frontier = deque([initial_node])
    visited = set()
    
    expanded = []
    current_frontier_size = 1
    maximum_frontier_size = 1
    
    while frontier:
        frontier_node = frontier.popleft()
        current_frontier_size -= 1
        print (frontier_node)
        actions = maze.actions(frontier_node)
        for child in frontier_node.expand(actions):
            if maze.is_goal(child):
                return (child,
                        expanded,
                        maximum_frontier_size,
                        len(expanded)+len(frontier))
            if child not in visited:
                expanded.append(child.state)
                
                visited.add(child)
                frontier.append(child)
                current_frontier_size += 1
                if current_frontier_size > maximum_frontier_size:
                    maximum_frontier_size = current_frontier_size
    raise Exception('Unable to find target')
bfs_results = breadth_first_search(maze)

<state:(x:8,y:10) path_cost:0 action:None>
<state:(x:8,y:11) path_cost:10 action:RIGHT>
<state:(x:7,y:10) path_cost:1 action:DOWN>
<state:(x:9,y:10) path_cost:1 action:UP>
<state:(x:9,y:11) path_cost:11 action:UP>
<state:(x:7,y:11) path_cost:11 action:DOWN>
<state:(x:8,y:12) path_cost:20 action:RIGHT>
<state:(x:8,y:10) path_cost:20 action:LEFT>
<state:(x:6,y:10) path_cost:2 action:DOWN>
<state:(x:8,y:10) path_cost:2 action:UP>
<state:(x:9,y:12) path_cost:21 action:RIGHT>
<state:(x:10,y:11) path_cost:12 action:UP>
<state:(x:8,y:11) path_cost:12 action:DOWN>
<state:(x:9,y:10) path_cost:21 action:LEFT>
<state:(x:6,y:11) path_cost:12 action:DOWN>
<state:(x:7,y:12) path_cost:21 action:RIGHT>
<state:(x:7,y:10) path_cost:21 action:LEFT>
<state:(x:8,y:13) path_cost:30 action:RIGHT>
<state:(x:8,y:11) path_cost:30 action:LEFT>
<state:(x:5,y:10) path_cost:3 action:DOWN>
<state:(x:7,y:10) path_cost:3 action:UP>
<state:(x:9,y:10) path_cost:3 action:UP>
<state:(x:9,y:13) path_cost:31 action:RIGHT>
<